In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from datasets import load_dataset
import os


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/anaconda3/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /root/anaconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /root/anaconda3/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/libbitsandbytes_cuda117.so...


In [2]:
model_id = "EleutherAI/polyglot-ko-12.8b"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

model = PeftModel.from_pretrained(model, './outputs/checkpoint-515/adapter_model/')

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [5]:
model.eval()
model.bfloat16()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [6]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

stop_words = ["</끝>"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])\

def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:", 
            return_tensors='pt', 
            return_token_type_ids=False
        ), 
        max_length=2048,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        stopping_criteria=stopping_criteria
    )
    print(tokenizer.decode(gened[0]))

In [9]:
gen('세계 2차대전에 대한 에세이를 작성하세요.')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 세계 2차대전에 대한 에세이를 작성하세요.

### 답변: 제1차 세계대전 발발 이후 영국 정부는 독일과의 모든 통신을 제한했습니다. 독일 정부는 보복을 위해 영국과의 모든 통신을 중단했습니다. 1939년, 히틀러는 "정부와 4번째 정부 총리"로부터 자신의 나라를 격리했습니다. 1940년 9월1일, 영국은 독일에 선전포고를 했고 6주 뒤에는 모든 전투에서 패배했습니다.*1891년 이후 일곱 나라가 사용한 무기의 총합이 1918년 세계 2차대전이 끝난 후 남은 무기의 총합에 비해 더 적었습니다.세계 2차 대전은 가장 큰 두 군인의 전투로 인한 사망자보다 전체 군인 사망자의 비율이 더 높은 전쟁입니다. 이 비율은 2:1에서 4:1 사이입니다.
이는 수백만 명의 죽음을 불러왔고 수백만 명의 사람들에게 파괴를 가져다주었습니다. 이 모든 피해는 승자와 패자 모두에게 엄청난 손실을 입힐 것이며, 그 결과는 우리 모두에게 장기간 영향을 미칠 것입니다.</끝>
